Working on Kinectrics Data

In [1]:
#%% 1. Library setup
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
DATANAME2 = "OfficeTest-Mesh_00.ply"
pcd = o3d.io.read_point_cloud("../Data/" + DATANAME2)

In [3]:
# Print basic info
print(pcd)

# Print number of points
print("Number of points:", np.asarray(pcd.points).shape[0])

# Show the first few points
print("First 5 points:\n", np.asarray(pcd.points)[:5])

PointCloud with 2499875 points.
Number of points: 2499875
First 5 points:
 [[ 6.375779 -7.119489 -1.878987]
 [ 6.371648 -7.118566 -1.881008]
 [ 6.371648 -7.119278 -1.878987]
 [ 6.375779 -7.118566 -1.88167 ]
 [ 6.379911 -7.119558 -1.878987]]


In [4]:
# shift point cloud to bypasss large coordinates approxximation
pcd_center = pcd.get_center()
print("pcd_center:", pcd_center)
print("Type:", type(pcd_center))

pcd_center: [ 4.5261486  -8.87957326 -1.13912606]
Type: <class 'numpy.ndarray'>


In [5]:
pcd.translate(-pcd_center)

PointCloud with 2499875 points.

In [6]:
o3d.visualization.draw_geometries([pcd])

In [7]:
# Random sampling
retained_ratio = 0.2 # keep 20% of the points
sampled_pcd = pcd.random_down_sample(retained_ratio)

In [8]:
o3d.visualization.draw_geometries([sampled_pcd], window_name = "Random Sampling")

In [9]:
# outlier removal
nn = 16 # How many neightbors are considered to calculate the averate distance for a given point
std_multiplier = 10 # threshold based on standard deviation. the lower the number, the more aggressive the filter will be

filtered_pcd, filtered_idx = pcd.remove_statistical_outlier(nn, std_multiplier)

In [10]:
# colour outliers in red
outliers = pcd.select_by_index(filtered_idx, invert=True)
outliers.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([filtered_pcd, outliers])

In [11]:
# voxel sampling
voxel_size = 0.05
pcd_downsampled = filtered_pcd.voxel_down_sample(voxel_size = voxel_size)

In [12]:
o3d.visualization.draw_geometries([pcd_downsampled])

In [13]:
#normals extraction
nn_distance = 0.05

In [14]:
radius_normals=nn_distance*4
pcd_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=16), fast_normal_computation=True)

In [15]:
pcd_downsampled.paint_uniform_color([0.6, 0.6, 0.6])
o3d.visualization.draw_geometries([pcd_downsampled,outliers])

In [16]:
# RANSAC
nn_distance = np.mean(pcd.compute_nearest_neighbor_distance())

In [17]:
distance_threshold = 0.1 # max distance a point can be from the plane model, and still be an inlier
ransac_n = 3 # number of points to sample for generating a plane model
num_iterations = 1000 # number of iterations

In [18]:
plane_model, inliers = pcd.segment_plane(distance_threshold=distance_threshold,ransac_n=3,num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: 0.01x + 0.00y + 1.00z + 0.70 = 0


In [19]:
inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)

#Paint the clouds
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])

#Visualize the inliers and outliers
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [20]:
# multi order RANSAC
# Empty dictionary to hold results of iterations
segment_models={} # Plane parameters
segments={} # Planar regions
max_plane_idx=10

In [21]:
# seperate inliers from outliers. Store inliers in segments dictionary.
rest=pcd
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(
    distance_threshold=0.1,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print("pass",i,"/",max_plane_idx,"done.")

pass 0 / 10 done.
pass 1 / 10 done.
pass 2 / 10 done.
pass 3 / 10 done.
pass 4 / 10 done.
pass 5 / 10 done.
pass 6 / 10 done.
pass 7 / 10 done.
pass 8 / 10 done.
pass 9 / 10 done.


In [22]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

In [9]:
# DBSCAN
epsilon = 0.3
min_cluster_points = 35

In [30]:
rest=sampled_pcd
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(
    distance_threshold=0.1,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    labels = np.array(segments[i].cluster_dbscan(eps=epsilon, min_points=min_cluster_points)) # DBSCAN clustering
    candidates=[len(np.where(labels==j)[0]) for j in np.unique(labels)]
    best_candidate=int(np.unique(labels)[np.where(candidates== np.max(candidates))[0]])
    rest = rest.select_by_index(inliers, invert=True) + segments[i].select_by_index(list(np.where(labels!=best_candidate)[0]))
    segments[i]=segments[i].select_by_index(list(np.where(labels== best_candidate)[0]))
    segments[i].paint_uniform_color(list(colors[:3]))
    print("pass",i,"/",max_plane_idx,"done.")


C:\Users\nchan\AppData\Local\Temp\ipykernel_15680\184731682.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  best_candidate=int(np.unique(labels)[np.where(candidates== np.max(candidates))[0]])


pass 0 / 10 done.
pass 1 / 10 done.
pass 2 / 10 done.
pass 3 / 10 done.
pass 4 / 10 done.
pass 5 / 10 done.
pass 6 / 10 done.
pass 7 / 10 done.
pass 8 / 10 done.
pass 9 / 10 done.


In [32]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

In [1]:
o3d.visualization.draw_geometries([rest])

NameError: name 'o3d' is not defined

In [10]:
labels = np.array(sampled_pcd.cluster_dbscan(eps=0.1, min_points=10)) # -1 is noise. 0-n are the clusters